# Unit Commitment

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.5.post1.dev0+g05ab0e7'

In [3]:
ams.config_logger(stream_level=10)

In [ ]:
sp = ams.load(ams.get_case('ieee14/ieee14_rted.xlsx'),
              setup=True,
              )

In [ ]:
sp.ED.run()

In [5]:
sp.UC.run()

Setup model of UC
UC data check passed.
- Generating symbols for UC
Set constrs pb: self.constrs["pb"]=cp.sum(self.routine.pd.v) - cp.sum(self.pg) == 0
Set constrs lub: self.constrs["lub"]=self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) - self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
Set constrs llb: self.constrs["llb"]=- self.routine.PTDF1.v @ (self.pg - self.routine.pd1.v) + self.routine.PTDF2.v @ self.routine.pd2.v - self.routine.rate_a.v <= 0
UC model set up in 0.0066 seconds.
UC solved as optimal in 0.0099 seconds with exit code 0.
